## 1. 라이브러리 import + CSV 불러오기

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import prince

from sklearn.preprocessing import  OrdinalEncoder
from sklearn.ensemble import ExtraTreesClassifier


In [3]:
train = pd.read_csv('./train.csv').drop(columns=['ID'])
test = pd.read_csv('./test.csv').drop(columns=['ID'])

In [4]:
X = train.drop('임신 성공 여부', axis=1) 
y = train['임신 성공 여부']

In [5]:
categorical_columns = [
    "시술 시기 코드",
    "시술 당시 나이",
    "시술 유형",
    "특정 시술 유형",
    "배란 자극 여부",
    "배란 유도 유형",
    "단일 배아 이식 여부",
    "착상 전 유전 검사 사용 여부",
    "착상 전 유전 진단 사용 여부",
    "남성 주 불임 원인",
    "남성 부 불임 원인",
    "여성 주 불임 원인",
    "여성 부 불임 원인",
    "부부 주 불임 원인",
    "부부 부 불임 원인",
    "불명확 불임 원인",
    "불임 원인 - 난관 질환",
    "불임 원인 - 남성 요인",
    "불임 원인 - 배란 장애",
    "불임 원인 - 여성 요인",
    "불임 원인 - 자궁경부 문제",
    "불임 원인 - 자궁내막증",
    "불임 원인 - 정자 농도",
    "불임 원인 - 정자 면역학적 요인",
    "불임 원인 - 정자 운동성",
    "불임 원인 - 정자 형태",
    "배아 생성 주요 이유",
    "총 시술 횟수",
    "클리닉 내 총 시술 횟수",
    "IVF 시술 횟수",
    "DI 시술 횟수",
    "총 임신 횟수",
    "IVF 임신 횟수",
    "DI 임신 횟수",
    "총 출산 횟수",
    "IVF 출산 횟수",
    "DI 출산 횟수",
    "난자 출처",
    "정자 출처",
    "난자 기증자 나이",
    "정자 기증자 나이",
    "동결 배아 사용 여부",
    "신선 배아 사용 여부",
    "기증 배아 사용 여부",
    "대리모 여부",
    "PGD 시술 여부",
    "PGS 시술 여부"
]

In [6]:
# 카테고리형 컬럼들을 문자열로 변환
for col in categorical_columns:
    X[col] = X[col].astype(str)
    test[col] = test[col].astype(str)

In [7]:
ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

X_train_encoded = X.copy()
X_train_encoded[categorical_columns] = ordinal_encoder.fit_transform(X[categorical_columns])

X_test_encoded = test.copy()
X_test_encoded[categorical_columns] = ordinal_encoder.transform(test[categorical_columns])

In [8]:
numeric_columns = [
    "임신 시도 또는 마지막 임신 경과 연수",
    "총 생성 배아 수",
    "미세주입된 난자 수",
    "미세주입에서 생성된 배아 수",
    "이식된 배아 수",
    "미세주입 배아 이식 수",
    "저장된 배아 수",
    "미세주입 후 저장된 배아 수",
    "해동된 배아 수",
    "해동 난자 수",
    "수집된 신선 난자 수",
    "저장된 신선 난자 수",
    "혼합된 난자 수",
    "파트너 정자와 혼합된 난자 수",
    "기증자 정자와 혼합된 난자 수",
    "난자 채취 경과일",
    "난자 해동 경과일",
    "난자 혼합 경과일",
    "배아 이식 경과일",
    "배아 해동 경과일"
]

In [9]:
X_train_encoded[numeric_columns] = X_train_encoded[numeric_columns].fillna(0)
X_test_encoded[numeric_columns] = X_test_encoded[numeric_columns].fillna(0)

## 2. EDA 모음

In [11]:

#'임신 시도 또는 마지막 임신 경과 연수' 열 제거
train = train.drop(columns=['난자 해동 경과일'])
test = test.drop(columns=['난자 해동 경과일'])

#'시술 유형' 열 제거
train = train.drop(columns=['PGS 시술 여부'])
test = test.drop(columns=['PGS 시술 여부'])

#'시술 유형' 열 제거
train = train.drop(columns=['PGD 시술 여부'])
test = test.drop(columns=['PGD 시술 여부'])


#'시술 유형' 열 제거
train = train.drop(columns=['착상 전 유전 검사 사용 여부'])
test = test.drop(columns=['착상 전 유전 검사 사용 여부'])

#'시술 유형' 열 제거
train = train.drop(columns=['임신 시도 또는 마지막 임신 경과 연수'])
test = test.drop(columns=['임신 시도 또는 마지막 임신 경과 연수'])

#'시술 유형' 열 제거
train = train.drop(columns=['배아 해동 경과일'])
test = test.drop(columns=['배아 해동 경과일'])


train과 test dataframe에 대해 지금까지의 EDA 관련 코드를 전부 모은 셀입니다
일일이 읽으려면 정신 나갈 수 있음..

In [13]:
# 정자 면역학적 요인 drop
train = train.drop('불임 원인 - 정자 면역학적 요인', axis=1)
test = test.drop('불임 원인 - 정자 면역학적 요인', axis=1)

# 남성 요인 MCA
features_male = ["불임 원인 - 남성 요인", "불임 원인 - 정자 농도", "불임 원인 - 정자 운동성", "불임 원인 - 정자 형태"]
subset_train = train[features_male].copy()  # 선택한 feature만 사용
subset_test = test[features_male].copy()

# prince 라이브러리로 MCA 모델 훈련 (n_components=1로 차원 축소)
mca = prince.MCA(n_components=1)

# MCA 모델을 훈련시키고 차원 축소된 데이터 얻기
mca_result_train = mca.fit_transform(subset_train)
mca_result_test = mca.fit_transform(subset_test)

# 기존 남성 요인 feature 삭제
train = train.drop(columns=features_male)
test = test.drop(columns=features_male)

# 차원 축소된 데이터를 원본 데이터에 새로운 열로 추가
train["불임 원인 - 남성 요인"] = mca_result_train
test["불임 원인 - 남성 요인"] = mca_result_test

In [14]:
import numpy as np
import pandas as pd

# 불임 원인 관련 feature 개수 카운트
male_causes = ['남성 주 불임 원인']
female_causes = ['여성 주 불임 원인']
couple_causes = ['부부 주 불임 원인']

train['남성 불임 원인 개수'] = train[male_causes].sum(axis=1)
test['남성 불임 원인 개수'] = test[male_causes].sum(axis=1)

train['여성 불임 원인 개수'] = train[female_causes].sum(axis=1)
test['여성 불임 원인 개수'] = test[female_causes].sum(axis=1)

train['부부 불임 원인 개수'] = train[couple_causes].sum(axis=1)
test['부부 불임 원인 개수'] = test[couple_causes].sum(axis=1)

# 기존 feature 제거
train = train.drop(columns=['남성 주 불임 원인', '여성 주 불임 원인', '부부 주 불임 원인'])
test = test.drop(columns=['남성 주 불임 원인', '여성 주 불임 원인', '부부 주 불임 원인'])

# 이제 각 데이터셋은 '남성 불임 원인 개수', '여성 불임 원인 개수', '부부 불임 원인 개수' 컬럼을 포함하게 됩니다.

In [15]:
import numpy as np
import pandas as pd

def infertility_cause(row):
    if row['부부 부 불임 원인']:
        return 'Both'
    elif row['남성 부 불임 원인']:
        return 'Male'
    elif row['여성 부 불임 원인']:
        return 'Female'
    else:
        return 'Unknown'

train['Infertility_Cause'] = train.apply(infertility_cause, axis=1)
test['Infertility_Cause'] = test.apply(infertility_cause, axis=1)

# 기존 컬럼 제거
train = train.drop(columns=['남성 부 불임 원인', '여성 부 불임 원인', '부부 부 불임 원인'])
test = test.drop(columns=['남성 부 불임 원인', '여성 부 불임 원인', '부부 부 불임 원인'])

# Label Encoding 또는 One-Hot Encoding 적용
train = pd.get_dummies(train, columns=['Infertility_Cause'])
test = pd.get_dummies(test, columns=['Infertility_Cause'])


In [16]:
# 여성 불임 관련 feature 개수 카운트
female_causes = ['불임 원인 - 난관 질환', '불임 원인 - 배란 장애', '불임 원인 - 자궁경부 문제', '불임 원인 - 자궁내막증']
train['여성 불임 원인 개수'] = train[female_causes].sum(axis=1)
test['여성 불임 원인 개수'] = test[female_causes].sum(axis=1)

# 기존 feature 유지 여부에 따라 선택적으로 제거 가능
train = train.drop(columns=['불임 원인 - 난관 질환', '불임 원인 - 배란 장애', '불임 원인 - 자궁경부 문제', '불임 원인 - 자궁내막증'])
test = test.drop(columns=['불임 원인 - 난관 질환', '불임 원인 - 배란 장애', '불임 원인 - 자궁경부 문제', '불임 원인 - 자궁내막증'])

## 4. 학습 (Lightgbm)

In [18]:
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

# 하이퍼파라미터 후보값 설정
param_dist = {
    'num_leaves': [31, 50, 70, 100, 150],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'n_estimators': [100, 200, 300, 500],
    'max_depth': [-1, 5, 10, 20],
    'subsample': [0.6, 0.7, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.7, 0.8, 1.0]
}

# RandomizedSearchCV 설정 (랜덤 탐색, cv=5)
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist, 
                                   n_iter=50, cv=5, n_jobs=-1, scoring='roc_auc', random_state=42)

# 모델 학습
random_search.fit(X_train_encoded, y)

# 최적의 하이퍼파라미터 출력
print(f"Best parameters found: {random_search.best_params_}")

# 최적의 모델로 예측 수행
best_model = random_search.best_estimator_

# 테스트 데이터에 대한 확률 예측
pred_proba = best_model.predict_proba(X_test_encoded)[:, 1]

# 샘플 제출 파일 로드 및 예측 확률 추가
sample_submission = pd.read_csv('./sample_submission.csv')
sample_submission['probability'] = pred_proba

# 결과를 CSV 파일로 저장
sample_submission.to_csv('./lightGBM_250218_randomized_optimized.csv', index=False)


NameError: name 'model' is not defined